# Evaluator-Optimizer 模式演示

本notebook演示了Evaluator-Optimizer（评估者-优化者）模式。
这是一种竞争性评估方法，多个LLM回答相同问题，然后由评审LLM进行排名。

**工作流程：**
1. 问题生成器LLM创建测试问题
2. 多个竞争LLM回答相同问题
3. 评审LLM评估所有答案并进行排名

**应用场景：**
- 模型性能对比
- 最佳答案筛选
- 质量评估系统

## 1、加载密钥环境OpenRouter

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

load_dotenv(override=True)
api_key = os.getenv("OPENROUTER_API_KEY")
if api_key:
    print("✓ API Key loaded successfully.")
else:
    print("✗ Failed to load API Key.")

## 2、初始化OpenRouter客户端

In [ ]:
openrouter = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=api_key)
print("✓ OpenRouter客户端初始化成功")

## 3、定义问题生成提示词

使用GPT模型生成用于评估商学硕士的商业问题

In [ ]:
# 构建问题生成提示
request = "请提出三条有难度的、简短的、非计算的现实商业的问题，用于评估商学硕士的智力水平。"
request += "请将每个问题编号，仅回答问题不做任何解释。"

messages = [{"role": "user", "content": request}]

print("=== 问题生成提示词 ===")
print(request)
print("\n提示词已准备完毕")

## 4、生成评估问题

使用 **gpt-5-mini** 生成商业评估问题

In [ ]:
print("\n=== 正在生成评估问题 ===")
print("使用模型: gpt-5-mini\n")

response = openrouter.chat.completions.create(
    model="gpt-5-mini",
    messages=messages
)

question = response.choices[0].message.content

print("="*70)
print("生成的评估问题:")
print("="*70)
display(Markdown(question))

## 5、配置竞争模型

定义三个参赛模型及其特点

In [ ]:
# 定义竞争模型配置
COMPETITOR_MODELS = [
    {
        "model": "gpt-5-nano",
        "name": "GPT-5 Nano",
        "description": "OpenAI的轻量级模型"
    },
    {
        "model": "anthropic/claude-sonnet-4-5",
        "name": "Claude Sonnet 4.5",
        "description": "Anthropic的平衡型模型"
    },
    {
        "model": "deepseek/deepseek-chat",
        "name": "DeepSeek Chat",
        "description": "DeepSeek的对话模型"
    }
]

print("=== 竞争模型配置 ===")
for idx, config in enumerate(COMPETITOR_MODELS, 1):
    print(f"{idx}. {config['name']} ({config['model']})")
    print(f"   {config['description']}")

# 初始化存储
competitors = []
answers = []
messages = [{"role": "user", "content": question}]

## 6、竞争模型1：GPT-5 Nano 回答

In [ ]:
print("\n" + "="*70)
print("竞争者 #1: GPT-5 Nano")
print("="*70)

model_config = COMPETITOR_MODELS[0]
model_name = model_config["model"]

response = openrouter.chat.completions.create(
    model=model_name,
    messages=messages
)

answer = response.choices[0].message.content
display(Markdown(answer))

competitors.append(model_name)
answers.append(answer)

print(f"\n✓ 回答长度: {len(answer)} 字符")

## 7、竞争模型2：Claude Sonnet 4.5 回答

In [ ]:
print("\n" + "="*70)
print("竞争者 #2: Claude Sonnet 4.5")
print("="*70)

model_config = COMPETITOR_MODELS[1]
model_name = model_config["model"]

response = openrouter.chat.completions.create(
    model=model_name,
    messages=messages
)

answer = response.choices[0].message.content
display(Markdown(answer))

competitors.append(model_name)
answers.append(answer)

print(f"\n✓ 回答长度: {len(answer)} 字符")

## 8、竞争模型3：DeepSeek Chat 回答

In [ ]:
print("\n" + "="*70)
print("竞争者 #3: DeepSeek Chat")
print("="*70)

model_config = COMPETITOR_MODELS[2]
model_name = model_config["model"]

response = openrouter.chat.completions.create(
    model=model_name,
    messages=messages
)

answer = response.choices[0].message.content
display(Markdown(answer))

competitors.append(model_name)
answers.append(answer)

print(f"\n✓ 回答长度: {len(answer)} 字符")

## 9、汇总所有竞争者的回答

展示所有模型的回答概览

In [ ]:
print("\n" + "="*70)
print("竞争者列表")
print("="*70)
for idx, competitor in enumerate(competitors, 1):
    print(f"{idx}. {competitor}")

print(f"\n总共 {len(competitors)} 个竞争模型参与评估")

## 10、合并所有回答供评审使用

In [ ]:
together = ""
for index, answer in enumerate(answers):
    together += f"来自于{index+1}号模型的回答:\n{answer}\n\n"

print("=== 合并后的回答 ===")
print(f"总字符数: {len(together)}")
print("\n" + together[:500] + "..." if len(together) > 500 else together)

## 11、构建评审提示词

准备评审LLM的提示词，要求其对所有答案进行排名

In [ ]:
judge = f"""你是一位商学教授，正在对 {len(competitors)} 个竞争者进行评估。
每个模型都被提出了这个问题:

{question}

你的工作是评估每个回答的清晰度和论证的强度，并按最好到最差的顺序对它们进行排名。 
使用 JSON 进行响应，并且仅使用 JSON，格式如下:
{{"results": ["最佳竞争对手号码", "第二最佳竞争对手号码", "第三最佳竞争对手号码", ...]}}

以下是每位参赛者的回应：

{together}

现在用 JSON 响应竞争对手的排名顺序，仅此而已。 不包含 Markdown 格式或代码块."""

print("=== 评审提示词已构建 ===")
print(f"提示词长度: {len(judge)} 字符")

judge_messages = [{"role": "user", "content": judge}]

## 12、评审LLM进行评估

使用 **gpt-5-mini** 作为评审，对所有回答进行排名

In [ ]:
print("\n" + "="*70)
print("评审LLM正在评估...")
print("="*70)
print("评审模型: openai/gpt-5-mini\n")

response = openrouter.chat.completions.create(
    model="openai/gpt-5-mini",
    messages=judge_messages,
)

results = response.choices[0].message.content

print("评审结果（原始输出）:")
print(results)

## 13、解析并展示最终排名

解析JSON结果并展示模型排名

In [ ]:
import re

# 尝试提取JSON
try:
    results_dict = json.loads(results)
except:
    # 如果包含markdown代码块，提取JSON部分
    json_match = re.search(r'\{[^}]+\}', results, re.DOTALL)
    if json_match:
        results_dict = json.loads(json_match.group())
    else:
        print("无法解析评审结果")
        results_dict = {"results": ["1", "2", "3"]}

ranks = results_dict["results"]

print("\n" + "="*70)
print("🏆 最终排名结果")
print("="*70)

medals = ["🥇", "🥈", "🥉"]

for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    medal = medals[index] if index < len(medals) else f"#{index+1}"
    print(f"{medal} Rank {index+1}: {competitor}")
    
print("\n" + "="*70)

## 14、可视化评估流程

In [ ]:
flow_diagram = """
```
┌─────────────────────────────────────────────────────────────┐
│                Evaluator-Optimizer 模式流程图                │
└─────────────────────────────────────────────────────────────┘

        ┌──────────────────────┐
        │  问题生成器 (GPT)    │
        │    生成评估问题      │
        └──────────┬───────────┘
                   │
                   ▼
        ┌──────────────────────┐
        │    评估问题          │
        └──────────┬───────────┘
                   │
       ┌───────────┼───────────┐
       │           │           │
       ▼           ▼           ▼
  ┌────────┐ ┌────────┐ ┌────────┐
  │ GPT-5  │ │Claude  │ │DeepSeek│
  │ Nano   │ │Sonnet  │ │ Chat   │
  └───┬────┘ └───┬────┘ └───┬────┘
      │          │          │
      └──────────┼──────────┘
                 │
                 ▼
        ┌──────────────────┐
        │  收集所有回答    │
        └────────┬─────────┘
                 │
                 ▼
        ┌──────────────────┐
        │  评审LLM (GPT)   │ ◄── 评估答案质量
        │   进行排名       │     清晰度和论证强度
        └────────┬─────────┘
                 │
                 ▼
        ┌──────────────────┐
        │  🏆 最终排名     │
        │  1️⃣ 2️⃣ 3️⃣        │
        └──────────────────┘
```

### Evaluator-Optimizer 模式的核心优势

1. **客观评估**
   - 多个模型回答相同问题
   - 独立评审LLM进行盲评

2. **性能对比**
   - 直观展示不同模型的优劣
   - 帮助选择最适合的模型

3. **质量保证**
   - 通过竞争机制提高回答质量
   - 识别最佳实践和答案模式

4. **持续优化**
   - 基于排名结果优化提示词
   - 改进模型选择策略

### 适用场景

- **模型选型**: 在多个模型中选择最适合的
- **质量控制**: 确保AI回答达到标准
- **性能基准**: 建立模型性能基准测试
- **A/B测试**: 对比不同版本的效果
"""

display(Markdown(flow_diagram))

## 15、保存评估结果

In [ ]:
# 构建完整的评估报告
evaluation_report = {
    "question": question,
    "competitors": [
        {
            "rank": idx + 1,
            "model": competitors[int(ranks[idx])-1],
            "answer": answers[int(ranks[idx])-1],
            "answer_length": len(answers[int(ranks[idx])-1])
        }
        for idx in range(len(ranks))
    ],
    "judge_model": "openai/gpt-5-mini",
    "raw_judgment": results
}

# 保存到JSON文件
with open('evaluation_results.json', 'w', encoding='utf-8') as f:
    json.dump(evaluation_report, f, ensure_ascii=False, indent=2)

print("✓ 评估结果已保存到 evaluation_results.json")
print(f"\n评估统计:")
print(f"- 参与模型数: {len(competitors)}")
print(f"- 总回答字符数: {sum(len(a) for a in answers)}")
print(f"- 评审模型: openai/gpt-5-mini")

# 显示排名摘要
print("\n📊 排名摘要:")
for idx, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"  {idx+1}. {competitor}")